## Robustness of NeuralHash to Some Standard Image Processing Transformations

In [9]:
from data import IMAGENETTE
import utils

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

In [10]:
folder_path = '../images/'
images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)

[INFO] Images saved


### Apply the image transformations to the Images

In [11]:
def flip_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Flip the image horizontally
    flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Save the flipped image with a new file name
    flipped_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")
    flipped_image.save(flipped_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")

In [12]:
def rotate_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Rotate the image by 90 degrees clockwise
    rotated_image = image.transpose(Image.ROTATE_90)

    # Save the rotated image with a new file name
    rotated_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")
    rotated_image.save(rotated_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")

In [13]:
original_paths, flipped_paths, rotated_paths = [], [], []
original_hashes, flipped_hashes, rotated_hashes = [], [], []
for file in sorted(os.listdir(folder_path)):
    if file.endswith(('.jpg', '.jpeg', '.png')):
        original_paths.append(os.path.join(folder_path, file))
        if original_paths[-1] is not None:
            original_hashes.append(utils.compute_hash(original_paths[-1]))
        
        flipped_paths.append(flip_image(folder_path, file))    
        if flipped_paths[-1] is not None:
            flipped_hashes.append(utils.compute_hash(flipped_paths[-1]))

        rotated_paths.append(rotate_image(folder_path, file))    
        if rotated_paths[-1] is not None:
            rotated_hashes.append(utils.compute_hash(rotated_paths[-1]))

In [14]:
l2_threshold = 20
hamming_threshold = 40

### Compute the Hamming Distances and L2 Distortion between the Hashes of the Flipped Sets of Images

In [15]:
# Original and flipped
success = []
for orig_path, flipped_path, orig_hash, flipped_hash in zip(original_paths, flipped_paths, original_hashes, flipped_hashes):
    orig_img, flipped_img = utils.load_img(orig_path), utils.load_img(flipped_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, flipped_hash, "hamming"), utils.distance(orig_img, flipped_img, "l2")
    print(f'{orig_path} & {flipped_path}\t Hamming Distance: {hamming_dist}\t L2 Dist: {l2_dist}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

../images/1.jpeg & ../images/1_flipped.jpeg	 Hamming Distance: 40	 L2 Dist: 152.63767588626047
../images/10.jpeg & ../images/10_flipped.jpeg	 Hamming Distance: 53	 L2 Dist: 132.2264993702668
../images/100.jpeg & ../images/100_flipped.jpeg	 Hamming Distance: 44	 L2 Dist: 161.71817512601686
../images/11.jpeg & ../images/11_flipped.jpeg	 Hamming Distance: 45	 L2 Dist: 93.02784054101683
../images/12.jpeg & ../images/12_flipped.jpeg	 Hamming Distance: 34	 L2 Dist: 143.2992140931186
../images/13.jpeg & ../images/13_flipped.jpeg	 Hamming Distance: 35	 L2 Dist: 77.87784784474785
../images/14.jpeg & ../images/14_flipped.jpeg	 Hamming Distance: 33	 L2 Dist: 186.81350782293288
../images/15.jpeg & ../images/15_flipped.jpeg	 Hamming Distance: 37	 L2 Dist: 135.34499668493
../images/16.jpeg & ../images/16_flipped.jpeg	 Hamming Distance: 42	 L2 Dist: 69.42129158062043
../images/17.jpeg & ../images/17_flipped.jpeg	 Hamming Distance: 33	 L2 Dist: 121.70429798340454
../images/18.jpeg & ../images/18_flipp

In [20]:
flipped_asr = sum(success)/len(success)
print(f'ASR: {100*flipped_asr:.2f}%')

ASR: 0.00%


### Compute the Hamming Distances and L2 Distortion between the Hashes of the Rotated Sets of Images

In [21]:
# Original and rotated
for orig_path, rotated_path, orig_hash, rotated_hash in zip(original_paths, rotated_paths, original_hashes, rotated_hashes):
    orig_img, rotated_img = utils.load_img(orig_path), utils.load_img(rotated_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, rotated_hash, "hamming"), utils.distance(orig_img, rotated_img, "l2")
    print(f'{orig_path} & {rotated_path}\t Hamming Distance: {hamming_dist}\t L2 Dist: {l2_dist}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

../images/1.jpeg & ../images/1_rotated.jpeg	 Hamming Distance: 50	 L2 Dist: 147.06508828484982
../images/10.jpeg & ../images/10_rotated.jpeg	 Hamming Distance: 49	 L2 Dist: 179.8332227926498
../images/100.jpeg & ../images/100_rotated.jpeg	 Hamming Distance: 52	 L2 Dist: 161.63155266172035
../images/11.jpeg & ../images/11_rotated.jpeg	 Hamming Distance: 43	 L2 Dist: 134.77423690274256
../images/12.jpeg & ../images/12_rotated.jpeg	 Hamming Distance: 51	 L2 Dist: 151.00527887581993
../images/13.jpeg & ../images/13_rotated.jpeg	 Hamming Distance: 38	 L2 Dist: 95.09500783524895
../images/14.jpeg & ../images/14_rotated.jpeg	 Hamming Distance: 42	 L2 Dist: 158.48739671449752
../images/15.jpeg & ../images/15_rotated.jpeg	 Hamming Distance: 51	 L2 Dist: 130.66021539755306
../images/16.jpeg & ../images/16_rotated.jpeg	 Hamming Distance: 48	 L2 Dist: 71.45827108152014
../images/17.jpeg & ../images/17_rotated.jpeg	 Hamming Distance: 44	 L2 Dist: 127.75291607755523
../images/18.jpeg & ../images/18_

In [22]:
rotated_asr = sum(success)/len(success)
print(f'ASR: {100*rotated_asr:.2f}%')

ASR: 0.00%
